In [ ]:
# # requirements: pip install requests beautifulsoup4 pandas lxml openpyxl
# import time, re, csv, json
# from urllib.parse import urljoin
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

In [ ]:
# # Preflight check: verify required libraries are installed
# import importlib
# import sys
# import subprocess

# REQUIRED = [
#     "requests",
#     "bs4",          # BeautifulSoup
#     "pandas",
#     "lxml",         # faster HTML parser
#     "openpyxl"      # Excel writing support for pandas
# ]

# missing = []
# for pkg in REQUIRED:
#     try:
#         importlib.import_module(pkg)
#     except ImportError:
#         print(f"❌ Missing: {pkg}")
#         missing.append(pkg)

# if missing:
#     print("\nInstalling missing packages...\n")
#     subprocess.check_call([sys.executable, "-m", "pip", "install", *missing])
# else:
#     print("✅ All required packages already installed and importable!")


In [ ]:

# BASE = "https://totalwarwarhammer.fandom.com"
# INDEX_URL = "https://totalwarwarhammer.fandom.com/wiki/Immortal_Empires_special_buildings"

# HEADERS = {
#     "User-Agent": "IE-special-buildings-scraper (for personal research; contact me if issues)"
# }

# SLEEP = 0.7  # be kind to the wiki

# def get_soup(url):
#     r = requests.get(url, headers=HEADERS, timeout=30)
#     r.raise_for_status()
#     return BeautifulSoup(r.text, "lxml")

# def find_master_table(soup):
#     """
#     Find the big landmarks table that has headers including:
#     Name | Type | Settlement | Province | Region | Starting faction
#     """
#     tables = soup.find_all("table")
#     candidate = None
#     for t in tables:
#         # collect normalized header text
#         headers = [th.get_text(" ", strip=True).lower() for th in t.select("thead th")] \
#                   or [th.get_text(" ", strip=True).lower() for th in t.select("tr th")]
#         head = " ".join(headers)
#         if all(k in head for k in ["name", "type", "settlement", "province", "region"]):
#             candidate = t
#             break
#     return candidate

# def parse_master_table(table):
#     rows = []
#     # header index map
#     header_cells = table.find("tr")
#     headers = [h.get_text(" ", strip=True).lower() for h in header_cells.find_all(["th","td"])]
#     # build index lookup
#     def col_index(name):
#         for i, h in enumerate(headers):
#             if name in h:
#                 return i
#         return None

#     idx_name = col_index("name")
#     idx_type = col_index("type")
#     idx_settlement = col_index("settlement")
#     idx_province = col_index("province")
#     idx_region = col_index("region")
#     idx_starting = col_index("starting")

#     for tr in table.find_all("tr")[1:]:
#         tds = tr.find_all(["td","th"])
#         if len(tds) < 5: 
#             continue
#         def cell(i):
#             return tds[i].get_text(" ", strip=True) if i is not None and i < len(tds) else ""

#         # link to building page
#         name_cell = tds[idx_name] if idx_name is not None else None
#         link = name_cell.find("a") if name_cell else None
#         name = (link.get_text(" ", strip=True) if link else cell(idx_name)).strip()
#         url = urljoin(BASE, link["href"]) if link and link.has_attr("href") else ""

#         rows.append({
#             "building_name": name,
#             "page_url": url,
#             "type": cell(idx_type),
#             "settlement": cell(idx_settlement),
#             "province": cell(idx_province),
#             "region": cell(idx_region),
#             "starting_faction": cell(idx_starting)
#         })
#     return rows

# def extract_effects_from_building(url):
#     """
#     Returns:
#       factions (list[str]),
#       effects_long (list of dicts: {building_name, faction, effect})
#     """
#     soup = get_soup(url)
#     title = soup.select_one("h1")
#     building_name = title.get_text(strip=True) if title else url

#     # Try to locate the "Building Effects" or "Effects" section.
#     # We will look for any h2 whose text contains 'Effects', then parse following h3 + UL groups until the next h2.
#     effects_root = None
#     for h2 in soup.select("h2"):
#         txt = h2.get_text(" ", strip=True).lower()
#         if "effects" in txt:  # matches "Building Effects" or "Effects"
#             effects_root = h2
#             break

#     factions = []
#     long_rows = []
#     if effects_root:
#         # iterate siblings after h2 until next h2
#         for sib in effects_root.find_all_next():
#             if sib.name == "h2":
#                 break
#             if sib.name in ("h3","h4"):
#                 faction = sib.get_text(" ", strip=True)
#                 # skip empty/anchors
#                 if not faction or faction.lower() in ("effects",):
#                     continue
#                 factions.append(faction)
#                 # collect bullets that immediately follow this heading, until the next heading of same or higher level
#                 ul = []
#                 cur = sib
#                 while True:
#                     cur = cur.find_next_sibling()
#                     if cur is None: 
#                         break
#                     if cur.name in ("h2","h3","h4"):
#                         break
#                     # bullets can be in <ul> or plain <p> lines
#                     for li in cur.select("li"):
#                         ul.append(li.get_text(" ", strip=True))
#                     # safety: some pages put single lines in <p>
#                     if cur.name == "p":
#                         text = cur.get_text(" ", strip=True)
#                         if text:
#                             ul.append(text)
#                 for eff in [e for e in ul if e]:
#                     long_rows.append({
#                         "building_name": building_name,
#                         "page_url": url,
#                         "faction": faction,
#                         "effect": eff
#                     })

#     # Optional: try to infer province from intro text if the master table missed it
#     province_guess = None
#     intro = soup.select_one("p")
#     if intro:
#         text = intro.get_text(" ", strip=True)
#         m = re.search(r"provincial capital of the ([^.]+)", text, flags=re.I)
#         if m:
#             province_guess = m.group(1).strip()

#     return building_name, list(dict.fromkeys(factions)), long_rows, province_guess

# def main():
#     print("Fetching index…")
#     index_soup = get_soup(INDEX_URL)
#     table = find_master_table(index_soup)
#     if table is None:
#         raise RuntimeError("Could not find the landmarks table on the index page.")
#     master_rows = parse_master_table(table)
#     print(f"Found {len(master_rows)} buildings in the master table.")

#     # Effects per building
#     effects_long = []
#     factions_map = {}  # building_name -> list of factions
#     for i, row in enumerate(master_rows, 1):
#         url = row["page_url"]
#         if not url:
#             continue
#         print(f"[{i}/{len(master_rows)}] {row['building_name']}")
#         try:
#             bname, factions, long_rows, province_guess = extract_effects_from_building(url)
#             factions_map[bname] = factions
#             effects_long.extend(long_rows)
#             if not row.get("province") and province_guess:
#                 row["province"] = province_guess
#         except Exception as e:
#             print("  -> error:", e)
#         time.sleep(SLEEP)

#     # attach factions list to master rows
#     for r in master_rows:
#         facs = factions_map.get(r["building_name"], [])
#         r["factions_can_build"] = ", ".join(facs)

#     # Save CSVs
#     bld_df = pd.DataFrame(master_rows)
#     eff_df = pd.DataFrame(effects_long)

#     bld_df.to_csv("buildings.csv", index=False)
#     eff_df.to_csv("building_effects.csv", index=False)

#     # Save Excel (two sheets)
#     with pd.ExcelWriter("immortal_empire_buildings.xlsx", engine="openpyxl") as xw:
#         bld_df.to_excel(xw, index=False, sheet_name="buildings")
#         eff_df.to_excel(xw, index=False, sheet_name="building_effects")

#     # Optional machine-friendly JSON (nested by building -> faction -> effects)
#     nested = {}
#     for row in master_rows:
#         nested[row["building_name"]] = {
#             "page_url": row["page_url"],
#             "type": row["type"],
#             "settlement": row["settlement"],
#             "province": row["province"],
#             "region": row["region"],
#             "starting_faction": row["starting_faction"],
#             "factions": {}
#         }
#     for _, g in eff_df.groupby(["building_name", "faction"]):
#         bname = g["building_name"].iloc[0]
#         faction = g["faction"].iloc[0]
#         nested[bname]["factions"].setdefault(faction, [])
#         nested[bname]["factions"][faction].extend(g["effect"].tolist())

#     with open("immortal_empire_buildings.json", "w", encoding="utf-8") as f:
#         json.dump(nested, f, ensure_ascii=False, indent=2)

#     print("Done. Wrote buildings.csv, building_effects.csv, immortal_empire_buildings.xlsx, immortal_empire_buildings.json")

# if __name__ == "__main__":
#     main()


In [4]:
import sys, subprocess, importlib

REQUIRED = ["requests", "bs4", "pandas", "lxml", "openpyxl", "tqdm"]
missing = []
for pkg in REQUIRED:
    try:
        importlib.import_module(pkg)
    except ImportError:
        missing.append(pkg)

if missing:
    print("Installing:", missing)
    subprocess.check_call([sys.executable, "-m", "pip", "install", *missing])

import os, re, time, json
from urllib.parse import urljoin
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Optional: progress bar
try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x  # fallback: no progress bar


Installing: ['tqdm']
Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
BASE = "https://totalwarwarhammer.fandom.com"
INDEX_URL = "https://totalwarwarhammer.fandom.com/wiki/Immortal_Empires_special_buildings"
HEADERS = {"User-Agent": "IE-special-buildings-scraper (personal research)"}
SLEEP = 0.7  # be kind to the wiki; raise if you hit throttling
TIMEOUT = 30
RETRIES = 3

def get_soup(url: str) -> BeautifulSoup:
    """GET with retries and parser fallback (lxml -> html.parser)."""
    last_err = None
    for attempt in range(1, RETRIES + 1):
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
            r.raise_for_status()
            html = r.text
            try:
                return BeautifulSoup(html, "lxml")
            except Exception:
                return BeautifulSoup(html, "html.parser")
        except Exception as e:
            last_err = e
            time.sleep(1.0 * attempt)
    raise last_err

def norm_name(s: str) -> str:
    """Normalize building names (fixes KeyError from curly quotes/NBSP/etc.)."""
    s = (s or "")
    s = s.replace("\u00A0", " ")
    s = s.replace("’", "'").replace("“", '"').replace("”", '"')
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def find_master_table(soup: BeautifulSoup):
    """
    Find the table that contains headers like:
    Name | Type | Settlement | Province | Region | Starting faction
    """
    tables = soup.find_all("table")
    for t in tables:
        headers = [th.get_text(" ", strip=True).lower() for th in t.select("thead th")] \
                  or [th.get_text(" ", strip=True).lower() for th in t.select("tr th")]
        if not headers:
            continue
        head = " ".join(headers)
        if all(k in head for k in ("name", "type", "settlement", "province", "region")):
            return t
    return None

def parse_master_table(table) -> list:
    rows = []
    header_cells = table.find("tr")
    headers = [h.get_text(" ", strip=True).lower() for h in header_cells.find_all(["th","td"])]
    def col_index(name):
        for i, h in enumerate(headers):
            if name in h:
                return i
        return None

    idx_name = col_index("name")
    idx_type = col_index("type")
    idx_settlement = col_index("settlement")
    idx_province = col_index("province")
    idx_region = col_index("region")
    idx_starting = col_index("starting")  # Starting faction

    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all(["td","th"])
        if len(tds) < 5:
            continue

        def cell(i):
            return tds[i].get_text(" ", strip=True) if i is not None and i < len(tds) else ""

        name_cell = tds[idx_name] if idx_name is not None else None
        link = name_cell.find("a") if name_cell else None
        name = (link.get_text(" ", strip=True) if link else cell(idx_name)).strip()
        url = urljoin(BASE, link["href"]) if link and link.has_attr("href") else ""

        rows.append({
            "building_name": name,
            "page_url": url,
            "type": cell(idx_type),
            "settlement": cell(idx_settlement),
            "province": cell(idx_province),
            "region": cell(idx_region),
            "starting_faction": cell(idx_starting),
        })
    return rows


In [6]:
def extract_effects_from_building(url: str):
    """
    Returns:
      building_name (str),
      factions (list[str]),
      effects_long (list of dicts: {building_name, page_url, faction, effect}),
      province_guess (str|None)
    """
    soup = get_soup(url)
    title = soup.select_one("h1")
    building_name = title.get_text(strip=True) if title else url

    # Locate "Effects" or "Building Effects"
    effects_root = None
    for h2 in soup.select("h2"):
        txt = h2.get_text(" ", strip=True).lower()
        if "effects" in txt:
            effects_root = h2
            break

    factions = []
    long_rows = []
    if effects_root:
        node = effects_root
        while True:
            node = node.find_next_sibling()
            if node is None or node.name == "h2":
                break
            # Factions are usually h3/h4 under Effects
            if node.name in ("h3", "h4"):
                faction = node.get_text(" ", strip=True)
                if faction and faction.lower() != "effects":
                    factions.append(faction)
                # Collect subsequent bullets/paragraphs until next h3/h4/h2
                cursor = node
                effects_texts = []
                while True:
                    cursor = cursor.find_next_sibling()
                    if cursor is None or cursor.name in ("h2","h3","h4"):
                        break
                    # bullets
                    for li in cursor.select("li"):
                        txt = li.get_text(" ", strip=True)
                        if txt:
                            effects_texts.append(txt)
                    # single paragraphs sometimes used
                    if cursor.name == "p":
                        txt = cursor.get_text(" ", strip=True)
                        if txt:
                            effects_texts.append(txt)
                for eff in effects_texts:
                    long_rows.append({
                        "building_name": building_name,
                        "page_url": url,
                        "faction": faction if factions else "",
                        "effect": eff
                    })

    # Optional: try to infer province from intro paragraph if master list missed it
    province_guess = None
    intro = soup.select_one("p")
    if intro:
        text = intro.get_text(" ", strip=True)
        m = re.search(r"provincial capital of the ([^.]+)", text, flags=re.I)
        if m:
            province_guess = m.group(1).strip()

    # de-dup factions while preserving order
    seen = set(); uniq_factions = []
    for f in factions:
        if f not in seen:
            uniq_factions.append(f); seen.add(f)

    return building_name, uniq_factions, long_rows, province_guess


In [7]:
print("Fetching index…")
index_soup = get_soup(INDEX_URL)
table = find_master_table(index_soup)
if table is None:
    raise RuntimeError("Could not find the landmarks table on the index page.")

master_rows = parse_master_table(table)
print(f"Found {len(master_rows)} buildings on the master list.")

# Save a checkpoint so you can resume later
MASTER_CSV = "buildings_master.csv"
pd.DataFrame(master_rows).to_csv(MASTER_CSV, index=False)
print(f"Saved master table -> {MASTER_CSV}")

Fetching index…
Found 288 buildings on the master list.
Saved master table -> buildings_master.csv


In [8]:
MASTER_CSV = "buildings_master.csv"
EFFECTS_PARTIAL = "building_effects.partial.csv"
EFFECTS_CSV = "building_effects.csv"

master_df = pd.read_csv(MASTER_CSV)
master_rows = master_df.to_dict("records")

# Determine which building pages are already scraped (resume)
already = set()
if os.path.exists(EFFECTS_PARTIAL):
    part_df = pd.read_csv(EFFECTS_PARTIAL)
    already = set(part_df["page_url"].dropna().tolist())
else:
    part_df = pd.DataFrame(columns=["building_name","page_url","faction","effect"])
    part_df.to_csv(EFFECTS_PARTIAL, index=False)

effects_long = []

print("Scraping per-building effects…")
for row in tqdm(master_rows):
    url = str(row.get("page_url") or "")
    if not url:
        continue
    if url in already:
        continue
    try:
        bname, factions, long_rows, province_guess = extract_effects_from_building(url)
        effects_long.extend(long_rows)
        # Update province in-memory if empty and we guessed one
        if (not str(row.get("province") or "").strip()) and province_guess:
            row["province"] = province_guess

        # Append incrementally to partial CSV to allow true resume
        if long_rows:
            pd.DataFrame(long_rows).to_csv(EFFECTS_PARTIAL, mode="a", header=False, index=False)
        else:
            # still mark as visited
            pd.DataFrame([{"building_name": bname, "page_url": url, "faction": "", "effect": ""}]).to_csv(
                EFFECTS_PARTIAL, mode="a", header=False, index=False
            )
    except Exception as e:
        print("  -> error on", url, ":", e)
    time.sleep(SLEEP)

# Consolidate (dedupe) partial into final
eff_df = pd.read_csv(EFFECTS_PARTIAL)
eff_df.drop_duplicates().to_csv(EFFECTS_CSV, index=False)
print(f"Saved effects -> {EFFECTS_CSV}")

# Save updated master with any province guesses we filled
pd.DataFrame(master_rows).to_csv(MASTER_CSV, index=False)


Scraping per-building effects…


 98%|█████████▊| 282/288 [06:17<00:07,  1.21s/it]

  -> error on nan : Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


 98%|█████████▊| 283/288 [06:23<00:14,  2.87s/it]

  -> error on nan : Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


...

  -> error on nan : Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


100%|█████████▉| 287/288 [06:50<00:05,  5.80s/it]

  -> error on nan : Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


100%|██████████| 288/288 [06:57<00:00,  1.45s/it]

Saved effects -> building_effects.csv


In [9]:
MASTER_CSV = "buildings_master.csv"
EFFECTS_CSV = "building_effects.csv"
BUILDINGS_CSV = "buildings.csv"

master_df = pd.read_csv(MASTER_CSV)
eff_df = pd.read_csv(EFFECTS_CSV)

# Map building -> factions (from effects section headings)
factions_map = {}
if not eff_df.empty:
    g = eff_df.groupby(["building_name", "faction"])
    for (bname, faction), _ in g:
        if pd.isna(faction) or not str(faction).strip():
            continue
        factions_map.setdefault(bname, set()).add(str(faction).strip())

# attach factions list to master rows
master_rows = master_df.to_dict("records")
for r in master_rows:
    facs = sorted(factions_map.get(r["building_name"], []))
    r["factions_can_build"] = ", ".join(facs)

pd.DataFrame(master_rows).to_csv(BUILDINGS_CSV, index=False)
print(f"Saved buildings -> {BUILDINGS_CSV}")


Saved buildings -> buildings.csv


In [10]:
BUILDINGS_CSV = "buildings.csv"
EFFECTS_CSV = "building_effects.csv"
XLSX = "immortal_empire_buildings.xlsx"

bld_df = pd.read_csv(BUILDINGS_CSV)
eff_df = pd.read_csv(EFFECTS_CSV)

try:
    import openpyxl  # noqa: F401
    with pd.ExcelWriter(XLSX, engine="openpyxl") as xw:
        bld_df.to_excel(xw, index=False, sheet_name="buildings")
        eff_df.to_excel(xw, index=False, sheet_name="building_effects")
    print(f"Saved Excel -> {XLSX}")
except Exception as e:
    print(f"⚠️ Skipping Excel export: {e}")


Saved Excel -> immortal_empire_buildings.xlsx


In [11]:
BUILDINGS_CSV = "buildings.csv"
EFFECTS_CSV = "building_effects.csv"
JSON_OUT = "immortal_empire_buildings.json"

bld_df = pd.read_csv(BUILDINGS_CSV)
eff_df = pd.read_csv(EFFECTS_CSV) if os.path.exists(EFFECTS_CSV) else pd.DataFrame(
    columns=["building_name","page_url","faction","effect"]
)

def row_key(building_name: str, page_url: str) -> str:
    # Prefer stable page_url; fallback to normalized name
    return (page_url or "").strip() or norm_name(building_name)

nested = {}

# Seed from master/buildings.csv
for _, r in bld_df.iterrows():
    k = row_key(r.get("building_name",""), r.get("page_url",""))
    nested[k] = {
        "building_name": r.get("building_name",""),
        "page_url": r.get("page_url",""),
        "type": r.get("type",""),
        "settlement": r.get("settlement",""),
        "province": r.get("province",""),
        "region": r.get("region",""),
        "starting_faction": r.get("starting_faction",""),
        "factions": {}
    }

# Merge effects; create shells for buildings not in master (prevents KeyError)
orphan_count = 0
if not eff_df.empty:
    if "page_url" not in eff_df.columns:
        eff_df["page_url"] = ""
    for (_, faction), g in eff_df.groupby(["building_name","faction"], dropna=False):
        bname = str(g["building_name"].iloc[0])
        page = str(g["page_url"].iloc[0]) if "page_url" in g.columns else ""
        k = row_key(bname, page)
        if k not in nested:
            orphan_count += 1
            print(f"⚠️ Creating shell for orphaned building: {bname}")
            nested[k] = {
                "building_name": bname,
                "page_url": page,
                "type": "",
                "settlement": "",
                "province": "",
                "region": "",
                "starting_faction": "",
                "factions": {}
            }
        nested[k]["factions"].setdefault(str(faction or ""), [])
        nested[k]["factions"][str(faction or "")].extend(
            [e for e in g["effect"].dropna().astype(str).tolist() if e.strip()]
        )

with open(JSON_OUT, "w", encoding="utf-8") as f:
    json.dump(nested, f, ensure_ascii=False, indent=2)

print(f"Saved JSON -> {JSON_OUT}")
if orphan_count:
    print(f"ℹ️ Note: {orphan_count} building(s) had effects but weren’t in the master table; shells created in JSON.")


AttributeError: 'float' object has no attribute 'strip'